In [1]:
import pandas as pd
from scipy.optimize import linprog
import numpy as np


In [2]:
# Load dataset
df = pd.read_csv("Stockist_data.csv")

In [3]:
# Define metadata columns
metadata_cols = ['Partner_id', 'Geography', 'Stockist_Type', 'Scheme_Type', 'Sales_Value_Last_Period', 
                 'Sales_Quantity_Last_Period', 'MRP', 'Growth_Percentage', 'Discount_Applied', 
                 'Bulk_Purchase_Tendency', 'New_Stockist', 'Feedback_Score']


In [4]:
# Identify product columns (excluding metadata)
product_cols = [col for col in df.columns if col not in metadata_cols]

# Transform data to have 'Product_id' column
df_melted = df.melt(id_vars=metadata_cols, value_vars=product_cols, 
                     var_name='Product_id', value_name='Has_Product')
df_melted = df_melted[df_melted['Has_Product'] == 1].drop(columns=['Has_Product']) # Aggregate sales volume per Partner-Product-Scheme combination
product_schemes = df_melted.groupby(["Partner_id", "Product_id", "Scheme_Type"]).agg({
    "Sales_Value_Last_Period": "sum",
    "Sales_Quantity_Last_Period": "sum"
}).reset_index()


In [5]:
# Prepare data for optimization
optimization_data = product_schemes[["Product_id", "Scheme_Type", "Sales_Value_Last_Period"]]

# Function to perform LP optimization for each product
def optimize_schemes(product_group):
    schemes = product_group["Scheme_Type"].unique()
    num_schemes = len(schemes)

    if num_schemes == 0:
        return [None, None, None]  # Ensure blank entries

    if num_schemes <= 3:
        return list(schemes) + [None] * (3 - num_schemes)

    c = -product_group.groupby("Scheme_Type")["Sales_Value_Last_Period"].sum().values
    bounds = [(0, 1) for _ in range(num_schemes)]
    res = linprog(c, bounds=bounds, method='highs', options={"disp": False})
    
    return [None, None, None]


In [6]:
# Apply optimization
optimized_schemes = optimization_data.groupby("Product_id").apply(optimize_schemes).reset_index()
optimized_schemes[["Scheme_1", "Scheme_2", "Scheme_3"]] = pd.DataFrame(optimized_schemes[0].tolist(), index=optimized_schemes.index)
optimized_schemes = optimized_schemes.drop(columns=[0])


In [7]:
# Group Partner_ids per product
partners_per_product = df_melted.groupby("Product_id")["Partner_id"].apply(list).reset_index()


In [8]:
# Merge with optimized schemes
final_optimized_output = partners_per_product.merge(optimized_schemes, on="Product_id", how="left")
final_optimized_output.to_csv("Top_Optimized_Schemes_with_LP.csv", index=False)
